# Introduction

Citibike is the leading bikesharing platform in New York, including the districts of Manhattan, Brooklyn, Queens & Jersey City. That said, the company is far from being profitable due to growing competition from other modes of transportation (e.g., scooters, electric bikes, etc.). According to a consulting firm hired a few months ago, the operations of Citibike are inefficient due to the lack of data-driven decision processes. If Citibike were only 4\% more efficient, it would become profitable. During peak hours (e.g., morning commute time), the bikesharing platform is particularly unreliable. Approximately 20\% of the dock stations face a "stock-out" situation, meaning that there are no bikes available. In such cases, commuters choose other modes of transportation. 

<img src="citi-bike.jpg" width="500">

### Your assignment
The consulting firm recommends using **optimization models** to improve the operations, especially during peak time. As part of this new strategy, you are recently hired as a lead data scientist of Citibike. You suspect that the inventory of bikes and the dock stations are not well positioned given the incoming demand at peak times. In this workshop, you will develop a data-driven optimization model to improve the positioning of inventory at peak time. In particular, you will formulate and implement integer programming models.

*Workshop instructions*: **Please carefully read all the instructions below:**
- The code cells are partially filled. You need to complete and execute the code in each cell. *You can consult any material posted on Canvas to help you get started with the syntax. You can ask questions to the TAs and discuss with your teammates.*
- Make sure to answer all the questions along the way in a detailed manner.
- This is an individual assignment; you should differentiate your coding style, answers, commenting, etc. from those of your teammates.
- Only Q1-Q3.6 are meant to be covered in class. You are free to start working on the other questions, but you should not exchange with your teammates on all remaining questions.

*Submission*: Your submission should include the following:
- *Notebook:* An html file along with a .ipynb file **(85 pt)**
- *Executive summary:* A 1 pager report summarizing all your findings (at least 11 pt font, 1 inch margin, pdf/doc format). This report should take the form of an executive summary that combines elements of your analysis and business recommendations. Supporting evidence can be provided in an appendix, or by referencing the questions of the workshop. The report will be evaluated along 3 dimensions: clarity, scientific validity, practical relevance. **(15 pt)**


# Q1. Visualize the data (15 pt)

## Load the data

Your colleague has generated data sets that contains all the needed information about the demand and available inventory of bikes during *one hour at peak time*. Your initial analysis will be based on the following data sets:

- `demand.csv`: describes the number of bikes rented at each dock station (during one peak hour),
- `starting_inventory.csv`: describes the initial inventory of bikes at each dock station (before peak hour),
- `distances.csv`: describes the distance between any two dock stations (in km unit).


Ideally, the inventory of bikes available the beginning of the peak hour should approximately match the expected amount of demand. For example, if you expect 5 user requests in station A, and 5 user requests in station B in the next hour, the number of bikes in station A and B should be roughly equal, otherwise there is an imbalance of inventory. Using spatial visualization, you will check if the initial inventory is balanced or imbalanced.

*Execute the cells below. Read carefully all the comments*

In [1]:
# Import various packages
import pandas as pd
import numpy as np
import folium # visualisation package for spatial data (plot of maps)
import seaborn as sns # general visualization package 
import matplotlib.pyplot as plt # general visualization package 
#next command allows you to display the figures in the notebook
%matplotlib inline     

In [2]:
## In case folium is not installed, run the command below and import folium again
# ! conda install -c conda-forge folium --yes

Next, you need to load the data sets as dataframes using the Pandas function `pd.read_csv()`.

In [3]:
starting_inventory = pd.read_csv("starting_inventory.csv", index_col=0) 
# "index_col=0" is used so that the dataframe is indexed by the first column, which is the dock station ID
demand = pd.read_csv("demand.csv", index_col=0)
distances = pd.read_csv("distances.csv", index_col=0)
distances.columns = list(map(lambda x: int(x),distances.columns.tolist())) # the column names (dock IDs) are converted into integers
replenishment = pd.read_csv("replenishment.csv", index_col=0)


Using the command `.describe()`, you can familiarize yourself with the format of the dataframes, and  check if there are missing entries.

In [4]:
# Insert your code here
starting_inventory.describe()

,count,latitude,longitude
count,689.000000,689.000000,689.000000
mean,30.724238,40.732527,-73.967780
std,70.308432,0.041163,0.024024
min,10.000000,40.655400,-74.025353
25%,10.000000,40.695807,-73.987030
50%,10.000000,40.728419,-73.968044
75%,19.000000,40.767100,-73.949450
max,739.000000,40.814394,-73.910651


In [5]:
demand.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,40.732527,-73.967780,27.648766
std,0.041163,0.024024,56.377839
min,40.655400,-74.025353,1.000000
25%,40.695807,-73.987030,5.000000
50%,40.728419,-73.968044,10.000000
75%,40.767100,-73.949450,26.000000
max,40.814394,-73.910651,428.000000


In [6]:
distances.describe()

,119,120,127,143,144,146,150,157,161,167,...,534,536,539,540,545,546,72,79,82,83
count,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,...,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000
mean,5.234788,5.818614,5.660433,5.815784,5.173995,5.896868,4.557984,6.009891,5.136558,4.691174,...,6.487351,4.529948,4.646942,4.676314,4.500302,4.725332,5.935885,5.695771,5.451519,5.886057
std,3.101918,3.240807,2.416317,3.396272,3.039626,2.765462,2.232180,3.456671,2.369193,2.011353,...,3.129782,1.880601,2.148153,2.001787,1.912251,2.026796,2.523133,2.677876,2.832379,3.535927
min,0.000000,0.000000,0.000000,0.000135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.778453,3.166575,3.980364,2.922870,2.795046,3.792634,2.907156,3.077351,3.575707,3.018884,...,3.896037,3.009274,3.131241,3.093796,2.985554,3.137539,4.043380,3.715493,3.254939,3.078088
50%,4.243954,5.492614,5.792542,5.194791,4.199160,5.789344,4.364615,5.426828,5.290044,4.733095,...,6.257024,4.590299,4.426917,4.846375,4.599683,4.905780,5.945542,5.633804,5.191993,5.067243
75%,7.926693,8.437574,7.444620,8.712969,7.752457,7.900162,5.982674,8.953698,6.833613,6.320707,...,8.895602,6.124998,6.031318,6.364107,6.043458,6.460421,8.018556,7.575516,7.511705,8.959315
max,12.087375,12.776908,10.356681,12.864326,11.919733,11.534052,9.753680,13.111354,9.844240,9.969009,...,12.871711,9.299113,9.929280,9.222172,8.738407,9.322635,11.311183,11.157840,11.482100,13.259187


In [7]:
replenishment.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,9.869376,-17.764877,6.513788
std,1.136245,2.045241,11.240249
min,0.000000,-18.000000,0.000000
25%,10.000000,-18.000000,1.000000
50%,10.000000,-18.000000,3.000000
75%,10.000000,-18.000000,7.000000
max,10.000000,0.000000,108.000000


In [8]:
starting_inventory.isnull().sum()

count        0
latitude     0
longitude    0
dtype: int64

In [9]:
demand.isnull().sum()

latitude     0
longitude    0
count        0
dtype: int64

In [10]:
distances.isnull().sum().sum()

0

In [11]:
replenishment.isnull().sum()

latitude     0
longitude    0
count        0
dtype: int64

### Q1.0. What is the distance between the dock stations "120" and "146"? What is the maximum inventory of bikes initially available over all dock stations?

In [12]:
distances.loc[120, 146]

5.7893437175538125

In [13]:
starting_inventory["count"].max()

739

*Insert answer here: The distance between dock stations 120 and 146 is 5.79. The maximum inventory of bikes initially available over all dock stations is 739.*


## Plot the heatmaps of demand and inventory

In the next cells, we will generate a heatmap of New York, representing the level of demand in different regions.

*The code is provided. Feel free to experiment with different parameters to better understand their respective roles.*

In [14]:
def generateBaseMap(default_location=[40.71, -73.96], default_zoom_start=12.5):
    '''
    This function generates a base map using the folium package
    
    Arguments:
    default_location -> latitude, longitude (a list)
    default_zoom_start -> level of zoom of the map (a positive number)
    '''
    base_map = folium.Map(location=default_location,control_scale=True, zoom_start=default_zoom_start)
    # 'Stamen Toner' is a type of map; by default the map is "OpenStreetMap"
    return base_map

The next cell will plot a heatmap of the demand.

In [15]:
# Import the HeatMap object constructor from the folium package
from folium.plugins import HeatMap 

# To begin, generate a base map, which will be edited next
base_map = generateBaseMap()

# The demand is converted into a list of t-uples, each describing one request point: (longitude, latitude, count)
# Copy the demand dataframe 
demand_agg = demand.copy()
# Rounding latitudes and longitudes to nearest 0.001 decimal
demand_agg.loc[:,['latitude', 'longitude']] = demand_agg.loc[:,['latitude', 'longitude']].round(3)
# Aggregating by latitude and longitude 
demand_agg= demand_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# Converting demand into a list
demand_list = demand_agg.reset_index().values.tolist()

# Add the desired heatmap to the base map
HeatMap(data= demand_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

Ideally, you would like to compare the expected demand to the initial inventory at the dock stations. To this end, you need to visualize the inventory of bikes initially available.

*By re-using the code above, construct a heatmap for the inventory of bikes in the city (`starting_inventory`). You need to edit the lines of code relative to the construction of the demand list.*

In [16]:
# Uncomment and fill the code below

from folium.plugins import HeatMap
base_map = generateBaseMap()
inventory_agg = starting_inventory.copy()
# # Create a line of code to round latitudes and longitudes
inventory_agg.loc[:,['latitude', 'longitude']] = inventory_agg.loc[:,['latitude', 'longitude']].round(3)
# # Create a line of code to aggregate the inventory by latitudes and longitudes
inventory_agg= inventory_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# # Convert inventory_agg to a list inventory_list (fill below)
inventory_list = inventory_agg.reset_index().values.tolist() 
HeatMap(data= inventory_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

### Q1.1. Are there apparent mismatches between demand and inventory as you compare the two heatmaps?

*Insert answer here:The two heatmaps suggest that there are most likely some differences between demand and inventory, but it is difficult to get a clear sense of the magnitude of these differences because the color gradients of the two maps have different scales as in each map the scale is determined by the maximum and minimum of supply and demand, respectively. Assuming the scales are roughly similar for the two heatmaps, it would seem that supply might be greater than demand near Grand Central Terminal and in Lower Manhattan, while demand might be greater than supply near the New York Penn Station.*



Your colleagues argue that that it is difficult to eyeball the differences between the two heatmaps. It would be preferable to have a single heatmap showing the "imbalance" between demand and inventory. Specifically, you would like to identify  areas where the initial bike inventory is either insufficient or excessive, compared to the amount of demand.

### Q1.2. How can you quantify  the degree of imbalance between demand and inventory? Create heatmaps for the newly defined imbalance metric(s) showing the areas with lack or excess of bikes.

*Hint:* Define two new metrics that quantify the imbalance between demand inventory and reuse the same code as above: 
- The first metric could capture the deficit of inventory at the stations where demand_agg > inventory_agg,
- The second metric could capture the excess of inventory at the stations where inventory_agg > demand_agg.

*Insert answer here: We can capture the deficit of inventory as Max(0, demand-inventory). On the other hand, we can capture excess inventory as Max(0, inventory-demand).*


In [17]:
# Uncomment and fill the code below

base_map = generateBaseMap()
deficit = np.maximum(0, demand_agg.loc[:,"count"]-inventory_agg.loc[:,"count"])
deficit_list = deficit.fillna(0).reset_index().values.tolist()
HeatMap(data= deficit_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map
#deficit

In [18]:
base_map = generateBaseMap()
excess = np.maximum(0, inventory_agg.loc[:,"count"]-demand_agg.loc[:,"count"])
excess_list = excess.fillna(0).reset_index().values.tolist()
HeatMap(data= excess_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map
#excess

### Q1.3. Add one more visualization of your own. Describe what it shows and what your learn from it in 1-2 lines

*Insert answer here: The map below shows the relative imbalance between inventory supply and demand, meaning that it shows the difference between inventory supply and demand as a percentage of demand, instead of the absolute difference between supply and demand. This visualization tells us that there's a lot of relative excess supply at the outskirts of New York. While it's proabably not an optimal short-term solution to relocate these bikes to Manhattan, it might be optimal in the long run.*


In [19]:
base_map = generateBaseMap()
imbalance = np.divide(inventory_agg.loc[:,"count"]-demand_agg.loc[:,"count"], demand_agg.loc[:,"count"])
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()
HeatMap(data= imbalance_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

# Q2. Optimal Rebalancing of Bike Inventory (20 pt)

**Matching supply with demand**: Given the imbalances between demand and inventory, Citibike runs frequent "rebalancing" operations, where bikes are relocated from a dock station having an excess of inventory to a station having insufficient inventory. These rebalancing operations are run in an ad-hoc fashion. You would like to construct an optimization model to rigorously rebalance the inventory before the beginning of the peak hour.

**The economics of rebalancing:** Rebalancing operators have to be paid \\$1 per km per relocated bike. There are no geographical restrictions on the rebalancing of bikes (from any station to any other station). After the rebalancing is performed, the amount of demand satisfied at each station will the minimum between the new initial inventory level and the total demand (we assume that there is no further replenishment during the peak hour). Each satisfied user request brings \\$4 of revenue, on average. Hence, when bikes are unavailable and user requests are unfulfilled, Citibike loses the revenue opportunity of \\$4 per user. 

*In what follows, you will formulate and implement the rebalancing problem as an integer program. Starting with the inventory levels of `starting_inventory`, you want to find the bikes relocations that maximize the total net revenue. For simplicity, you can assume that the rebalancing operations happen instantaneously before the beginning of the peak hour. There is no increase in the supply of bikes during the peak hour.*

In [20]:
# The following lines of code import the gurobi package
import gurobipy as gp
from gurobipy import GRB,quicksum

## Model creation

### Q2.1. Create a GUROBI new model "m", named "rebalancing"

In [21]:
#Insert your code here:
m = gp.Model("rebalancing")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-30


## Decision variables

Clearly your main decisions have to do with the "relocation" of bikes.

In [22]:
# Instantiate the list of all stations
stations = demand.index.tolist()

# Next, we create am "integer" decision variable for each pair of dock stations (A,B)
relocation = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation")

### Q2.2. Explain in detail what the code in the cell above is doing.

*Insert your answer here: The first line of code creates a list of all the docking stations by pulling it from the demand dataset. The second line of code defines the decision variables of the optimization model as the number of bikes that are relocated between each pair of two existing docking stations in New York.*

Your colleague thinks that additional decision will be needed for the problem. She suggests the following line of code:

In [23]:
satisfied_demand = m.addVars(stations,vtype=GRB.INTEGER, lb = 0, name="satisfied_demand")

### Q2.3. Why are the variables `satisfied_demand` needed? How do we refer to such variables (which do not describe actual decisions)?

*Insert your answer here: The set of variables satisfied_demand represent the satisfied demand at all of New York's docking station, which is the minimum between the post-rebalancing supply and demand. This is not a decision variable but rather a variable that helps with modelling. This variable is needed because otherwise the objective function would be much more complicated to solve, as it would need to include the minimums.*

## Constraints

### Q2.4. Is there an upper bound on the number of bikes that can be relocated from one station to another? Incorporate the upper bound into the model.
*Hint: Use `starting_inventory.loc[i,"count"]` to access the initial number of bikes at station `i`.*

*Insert your answer here: The upper bound on the number of bikes that can be relocated from one station to all the other stations in New York is the initial inventory at that station.*

In [25]:
#Insert your code here:
m.addConstrs((quicksum(relocation[i, j] for j in stations)<=starting_inventory.loc[i,"count"] for i in stations), 
             name="maximum_relocation");


The satisfied demand in each station is the minimum between the demand and the inventory after rebalancing. For example, if station A has 10 bikes before the relocation and we relocate 2 extra bikes, we can satisfy up to 12 users. If the demand at station A is 8, we satisfy all 8 users. If the demand at station A is 15, we can only satisfy 12 users, and we lose 3 user requests. In general, we have the equation:

$$ {\rm satisfiedDemand}(A) ={\rm minimum} \left\{ {\rm InventoryAfterRebalancing}(A), {\rm Demand}(A)\right\}$$

Below, you will add two ensembles of constraints to our model `m` to capture the notion of satisfied demand.

### Q2.5. Add constraints imposing that `satisfied_demand` is smaller or equal to the demand, in each station.

In [26]:
#Insert your code here:
m.addConstrs((satisfied_demand[i]<=demand.loc[i, "count"] for i in stations),
                 name="satisfied_demand_vs_demand");

### Q2.6. Explain what the additional constraints below on the `satisfied_demand`  are doing. Why they are needed?

*Insert your answer here: This constraint states that a docking station's satisfied demand must be equal to or lower than the post-rebalancing supply. This constraint binds if the post-rebalancing supply at a docking station is lower than demand.*

In [27]:
# Additional constraints
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"]
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations)
                  for i in stations), name = "satisfied_demand_vs_inventory_after_rebalancing");


## Objective

### Q2.7. How would you formulate the net revenue as a linear expression? Specify the objective of the model.

*Hint: Recall that the objective function is specified using:* `m.setObjective(EXPRESSION,GRB.MAXIMIZE)`

Insert your answer here: The net revenue as a linear expression is:

average_revenue_per_ride * Sum(satisfied_demand[i] for i in stations) - Sum(relocation[i,j] * distances.loc[i, j] * relocation_cost_per_km for j in stations for i in stations) = 4 * Sum(satisfied_demand[i] for i in stations) - Sum(relocation[i,j] * distances.loc[i, j] for j in stations for i in stations)

The objective of te model is to maximize the net revenue.


In [28]:
#Insert your code here:
m.setObjective(4*quicksum(satisfied_demand[i] for i in stations)-quicksum(relocation[i,j]*distances.loc[i, j] for j in stations 
                                                                         for i in stations),GRB.MAXIMIZE)

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue/

In [29]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2756 rows, 475410 columns and 1898884 nonzeros
Model fingerprint: 0x36210a7d
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 62468.000000
Presolve removed 1378 rows and 689 columns (presolve time = 5s) ...
Presolve removed 1378 rows and 19978 columns
Presolve time: 7.39s
Presolved: 1378 rows, 455432 columns, 1337358 nonzeros
Variable types: 0 continuous, 455432 integer (6671 binary)
Found heuristic solution: objective 62739.189353
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barr

### Q2.8. Read the output of the optimization: What is the optimal net revenue?

*Insert your answer here: The optimal net revenue is $71314.1.* 



### Q2.9. How much revenue does Citibike gain using the optimal rebalancing compared to no rebalancing? Is this a significant increase? Carefully explain how you obtain the answer (feel free to add code below).
*Hint: The goal here is to quantity how the revenue changes after the optimal rebalancing vs. no rebalancing. To compute the revenue in the absence of rebalancing, you could reuse our optimization model with a small modification of the variables `relocation`.*

In [30]:
# The quickest way to get net revenue without rebalancing is to calculate satisfied_demand 
# for each docking station with the initial inventory supply, sum satisfied demand across
# all NY stations and multiply by 4.
4*np.minimum(starting_inventory["count"], demand["count"]).sum()

62468

In [31]:
# Alternatively, we can add a constraint in our model which specifies that all relocation must be equal to 0.
# In this case, the optimal net revenue is the net revenue without rebalancing.

no_rebalancing=m.addConstrs((relocation[i,j]==0 for i in stations for j in stations),name="no_rebalancing")

In [32]:
m.optimize()
printSolution()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 477477 rows, 475410 columns and 2373605 nonzeros
Model fingerprint: 0xc4a42dc1
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint no_rebalancing[119,317] by 8.000000000

Found heuristic solution: objective 62468.000000
Presolve removed 477477 rows and 475410 columns
Presolve time: 0.38s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.79 seconds (0.59 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 624

In [33]:
m.remove(no_rebalancing)

Insert your answer here: Without using rebalancing, Citibike makes a net revenue of 62468, while with optimal rebalancing the net revenue becomes 71314.1. This means that net revenue increases by 8846.1, or 14.16%. This looks like a significant increase.



The relocations are visualized on the heatmap in the next cell. 

*Code is provided, Execute the next cells. Feel free to vary the parameters to understand their role.*

In [35]:
from collections import namedtuple
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing
def get_arrows(locations, color='black', size=4, n_arrows=3,opacity = 1):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, color = color, number_of_sides=3, 
                      radius=size, rotation=rotation,opacity = opacity))
    return arrows

base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list,radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map       

### Q2.10. How do you interpret the plot? Is this what you would expect?

*Insert your answer: The plot seems to suggest that bikes are relocated to stations with the highest relative deficits (areas higher on the color spectrum) from stations that have the lowest relative deficits (areas lower on the color spectrum). The latter stations must actually have a negative relative deficit (i.e. a positive relative excess), as it wouldn't make economic sense to relocate from a station that has a deficit. We can see that bikes are often relocated between stations that are quite far away from each other, but this isn't always the case. This makes sense if deficits can't be covered from stations that are close by. Economically, the maximum distance we would be willing to relocate a bike at is 4km, since we would pay $4 for that, the same as the revenue we would get from the renting it out. The length of the largest arrows, which appears to be around 2.5-3km, seems to support this economic intuition.* 

# Q3. Rebalancing with replenishment (20 pt)

You manager has a concern about your analysis in Q2:

*I like the idea of rebalancing bikes based on an optimization model. However, you have omitted an important aspect of the problem. When a user completes her current trip, she returns the bike to the system. A new bike is made available at the destination station. You don't account for this in your current inventory, but this organic replenishment of bikes might be very helpful since it increases the number of bikes available at the destination station!*

The goal of Q3 is to update the model accordingly. You will reuse most of the existing code, with minor modifications.  Your analysis will be based on the following additional data set:

- `replenishment.csv`: describes the number of extra bikes that will be made available at each station after current users complete their trips. 

**For simplicity, we will assume that the replenishment happens instantaneously before the beginning of the peak hour.**

### Q3.1. Based on your manager's comment, did you under-estimate or over-estimate the inventory at each station in the previous question Q2?

*Insert your answer here: Based on the manager's comment, it seems that we have under-estimated the inventory at each station since replenishments will add a non-negative number of bikes to each station.*

## Loading the data

In [36]:
replenishment = pd.read_csv("replenishment.csv", index_col=0)

*You can visualize the replenishment data using `.describe()`.*

In [37]:
replenishment.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,9.869376,-17.764877,6.513788
std,1.136245,2.045241,11.240249
min,0.000000,-18.000000,0.000000
25%,10.000000,-18.000000,1.000000
50%,10.000000,-18.000000,3.000000
75%,10.000000,-18.000000,7.000000
max,10.000000,0.000000,108.000000


## Model creation

### Q3.2 Create the model object

*Hint: Reuse the same code as Q2.1.*

In [38]:
#Insert your code here:
m = gp.Model("rebalancing")

## Decision variables

### Q3.2. Add the decision variables to `m`
*Hint: Reuse the same code as Q2.2 and Q2.3*

In [39]:
#Insert your code here:
relocation = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation")
satisfied_demand = m.addVars(stations,vtype=GRB.INTEGER, lb = 0, name="satisfied_demand")

## Constraints

### Q3.3. Add an upper bound on the number of bikes that can be relocated from one station to another.
*Hint: Reuse the same code as Q2.4*

In [41]:
#Insert your code here:
m.addConstrs((quicksum(relocation[i, j] for j in stations)<=
              (starting_inventory.loc[i,"count"]) for i in stations), 
             name="maximum_relocation");


### Q3.4. Add constraints requiring that `satisfied_demand` is smaller or equal to the demand.
*Hint: Reuse the same code as Q2.5*

In [42]:
#Insert your code here:
m.addConstrs((satisfied_demand[i]<=demand.loc[i, "count"] for i in stations),
                 name="satisfied_demand_vs_demand");

 ### Q3.5. The `satisfied_demand` should be also be related to inventory. How would you modify the constraint of Q2.6 to account for replenishment? Add these constraints to the model.
 
 *Hint: You should use the same constraint as Q2.6, but also incorporate the replenishment quantity. For simplicity, you can assume that all the replenishment happens before the demand arrives. This assumption is reasonable if the system is "stationary".*

*Insert your answer here: Because we can assume that all the replenishment happens before the demand arrives, we can assume that the post-rebalancing inventory supply is starting_inventory[i]+sum(relocation[j,i] for j in stations)-sum(relocation[i,j] for j in stations)+replenishment[i]. Other than this, the constraint stays the same, stating that satisfied demand must be lower than the post-rebalancing inventory supply.



In [43]:
#Insert your code here:
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"]
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations) 
              + replenishment.loc[i,"count"] for i in stations),
             name = "satisfied_demand_vs_inventory_after_rebalancing");

## Objective

### Q3.6. What is the objective function? Specify the objective of the model `m`.

*Hint: Reuse the code of Q2.7*

In [44]:
#Insert your code here:
m.setObjective(4*quicksum(satisfied_demand[i] for i in stations)-quicksum(relocation[i,j]*distances.loc[i, j] for j in stations 
                                                                         for i in stations),GRB.MAXIMIZE)

*Insert your answer here: The objective function stays the same as in Q2.* 


## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue.

In [45]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2756 rows, 475410 columns and 1898884 nonzeros
Model fingerprint: 0x63903ce9
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 72024.000000
Presolve removed 1435 rows and 35425 columns (presolve time = 5s) ...
Presolve removed 1411 rows and 35401 columns
Presolve time: 5.66s
Presolved: 1345 rows, 440009 columns, 1281864 nonzeros
Variable types: 0 continuous, 440009 integer (3985 binary)
Found heuristic solution: objective 72164.331558
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root ba

### Q3.7. Read the output of the optimization: What is the optimal net revenue? Is it greater or smaller than that obtained without replenishment in question Q.2.8? Why?

*Insert your answer here: The optimal net revenue is now 75662.9. This is greater than the net revenue obtained without replenishment in Q.2.8. Net revenue increases for multiple reasons. First of all, for the docking stations which were initially below capacity (starting_inventory<demand) but would get to capacity after rebalancing in Question 2, positive replenishments to these docking stations increase net revenue because less relocations have to be made to these stations to get to capacity, which decreases costs. Second, for the docking stations which were initially below capacity and woulnd't get to capacity even after rebalancing in Question 2, positive replenishments to these docking stations increase net revenue due to direct revenue increase. Lastly, even if positive replenishments are made to a docking station which is initially at or above capacity, net revenue can still increase as a result of this because more bikes could be relocated from this station to a nearby station that has a deficit before rebalancing, either by helping to fill the remaining deficit from Q2, or by replacing bikes which are relocated from further away at a higher cost.*


### Q3.8. How much revenue does Citibike gain using the optimal rebalancing compared to no rebalancing? How does this compare to Q2.9? Why?
*Hint: The goal here is to quantity the revenue change after vs. before the optimal rebalancing. To compute the revenue in the absence of rebalancing, you could reuse our optimization model with a very small modification of the variables `relocation`.*

In [46]:
4*np.minimum(starting_inventory["count"]+replenishment["count"], demand["count"]).sum()

72024

*Insert your answer here: Without using rebalancing and with replenishments, Citibike makes a net revenue of 72024, while with optimal rebalancing and replenishments the net revenue becomes 75662.9. This means that net revenue increases by 3638.9, or 5.05%. Thus, the increase in net revenue is smaller compared to Q2.9. This makes sense because, assuming replenishments are randomly distributed between stations, stations that had dificits in Q2 may have smaller deficits or no deficits at all after replenishments, and therefore fewer relocations are needed from other stations.*



The relocations are visualized on the heatmap in the next cell. *Execute the next cell.*

In [47]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance ratio between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map                

### Q3.9. How do you interpret the plot? Is this what you would expect compared to Q2.10?

*Insert your answer: We can clearly see that in this plot arrows are much shorter, fewer, and more transparent than in the corresponding plot from Q2, meaning that fewer bikes are being relocated between stations and that they are being relocated from less far away. This perfectly supports the intuition outlined in the previous question.* 

# Q4. Vehicle Capacity (10 pt)

In reality, Citibike can utilize two types of vehicles:
- Large capacity vehicles (LCV): These are the default vehicles. They can carry any arbitrary number of bikes. Due to the large size of the vehicle, rebalancing operators have to be paid \\$1 per km per relocated bike (similar to Q2 and Q3).
- Small capacity vehicles (SCV): These specialized vehicles can handle up to 10 bikes per rebalancing operation. Rebalancing operators have to be paid \\$0.5 per km per relocated bike.

For each relocation operation (corresponding to a pair of stations), the firm needs to choose exactly one type of vehicle. The goal of Q4 is to update the model constructed in Q3 to incorporate the following logical condition: *EITHER we use a small capacity vehicle and the number of relocated bikes is less than 10 OR we use a large capcity vehicle*.

## Decision variables: Small vs. large vehicles

We add relocation variables corresponding to the use of small vs. large vehicles.

In [48]:
# The decision variables describe the number of bikes relocated from station A to station B
relocation_SCV = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_small")
relocation_LCV  = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_large")

## Constraints

In the next constraint, we impose that the total number of relocations is the sum of relocations using SCVs and LCVs.

In [49]:
m.addConstrs(relocation[i,j] == relocation_SCV[i,j] +relocation_LCV[i,j] for i in stations for j in stations);

### Q4.1. Add constraints requiring that EITHER we use an SCV with at most 10 bikes OR we don't use an SCV 
*Hint: Use the big-M method covered in today's class. Define a binary auxiliary variable for each pair of stations to switch between the two conditions.*

In [50]:
# Uncomment and fill the code below:

M = 10000
auxiliary = m.addVars(stations,stations,vtype=GRB.BINARY, name="OR_auxiliary")
m.addConstrs(relocation[i,j] <=relocation_SCV[i,j] + (1-auxiliary[i, j])*M for i in stations for j in stations);
m.addConstrs(relocation_SCV[i,j] <=10*auxiliary[i, j]  for i in stations for j in stations);

## Objective

The objective function is modified accordingly. *Execute the cells below.*

In [51]:
# Set the objective: maximize the net revenue
m.setObjective(4*satisfied_demand.sum()
               -1*quicksum(relocation_LCV[i,j]*distances.loc[i,j] for i in stations for j in stations)
               -0.5*quicksum(relocation_SCV[i,j]*distances.loc[i,j] for i in stations for j in stations)               
               ,GRB.MAXIMIZE)

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue.

In [52]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1426919 rows, 1899573 columns and 5696652 nonzeros
Model fingerprint: 0x75c650b9
Variable types: 0 continuous, 1899573 integer (474721 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [7e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective 75752.5 (4.75s)
Loaded MIP start from previous solve with objective 75752.5
Processed MIP start in 4.88 seconds (1.56 work units)

Presolve removed 1411 rows and 476788 columns (presolve time = 5s) ...
Presolve removed 71590 rows and 544933 columns (presolve time = 10s) ...
Presolve removed 348712 rows and 544933 columns (presolve time = 15s) ...
Presolve removed 348712 rows and 544933 co

The relocations are visualized on the heatmap in the next cell. *Execute the next cell.*

In [53]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map                

### Q4.2. How do you interpret the output compared to Q3.7-Q3.9? Does the net revenue increase or decrease? 

*Insert your answer: The optimal net revenue is now 75881.9, which is 219 greater than 75662.9. It woud have beeen impossible for net revenue to be lower in this scenario because we simply add another option for the relocation operation, without restricting the possibility to choose the former option and have the same net revenue as before. The 219 net revenue increase is most likely caused by small capacity vehicles (SCVs) replacing large capacity vehicles (LCVs) in relocation operations that involve a small number of bikes, where SCVs clearly have a cost advantage. Looking at the two maps, we can see that compared to the map in 3.9, this map has some additional arrows, which probably represent SCV relocations. These arrows are usually more transparent and longer, which makes sense as the maximum capacity for these relocations is 10 and as SCV relocations can make economical sense over a longer distance, due to the low cost per km.* 


# Q5. Procurement of vehicles (5pt)

In reality, Citibike does not operate the SCVs and LCVs, which are booked through an external provider. There are various costs and constraints associated with the procurement of  small and large vehicles. Specifically, we have
- *Fixed cost for small vehicles:* You should pay a fixed cost of 80\$ to be able to use small vehicles. Note that this cost does **not** scale with the number of small vehicles.
- *Per vehicle cost:* You should pay a variable cost of 10\$ per vehicle.
- *Number of rebalancing operations per vehicle:* Suppose that each vehicle can perform at most 3 distinct rebalancing operations witin the imparted time.

How would you modify the model to incorporate procurement costs? **Build a new model that captures all the above requirements.**

In [18]:
#Insert your code here:
m = gp.Model("rebalancing")

#binary decision variable for whether we pay for the right to be able to use small vehicles
get_small_vehicles = m.addVar(vtype=GRB.BINARY, name="get_small_vehicles")

#integer decision variable for the number of large vehicles rented
number_small_vehicles = m.addVar(vtype=GRB.INTEGER, name="number_small_vehicles")

#integer decision variable for the number of small vehicles rented
number_large_vehicles = m.addVar(vtype=GRB.INTEGER, name="number_large_vehicles")

#binary decision variable for whether we relocate from station i to j using a small vehicle
is_small_relocation = m.addVars(stations,stations,vtype=GRB.BINARY, lb = 0, name="is_small_relocation")

#binary decision variable for whether we relocate from station i to j using a large vehicle
is_large_relocation = m.addVars(stations,stations,vtype=GRB.BINARY, lb = 0, name="is_large_relocation")

#constraint that states that the maximum number of relocations using small vehicles is 3*number_small_vehicles*get_small_vehicles
max_small_relocations = m.addConstr(quicksum(is_small_relocation[i,j] for i in stations for j in stations)<=3*number_small_vehicles* 
                                    get_small_vehicles,name="max_small_relocations")

#constraint that states that the maximum number of relocations using large vehicles is 3*number_large_vehicles
max_large_relocations = m.addConstr(quicksum(is_large_relocation[i,j] for i in stations for j in stations)<=3*number_large_vehicles,
                                   name="max_large_relocations")

#constraint that states that relocation from station i to j can't be made by both a small and a large vehicle
either_SCV_or_LCV =  m.addConstrs((is_small_relocation[i,j]*is_large_relocation[i,j]==0 for i in stations for j in stations),
                                  name="either_SCV_or_LCV")

#integer decision variable for the number of bikes relocated from station i to j using small vehicles
relocation_SCV = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_small")

#integer decision variable for the number of bikes relocated from station i to j using large vehicles
relocation_LCV  = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_large")

#constraint which states that the number of bikes relocated from station i to j using small vehicles can be positive only
#if we relocate from station i to j using a small vehicle, and that if we do, the maximum number of relocated bikes is 10
relocation_SCV_positive_if_small_is_1=m.addConstrs((relocation_SCV[i, j]<=10*is_small_relocation[i, j] 
                                        for i  in stations for j in stations), name="relocation_SCV_positive_if_small_is_1")

#constraint which states that the number of bikes relocated from station i to j using large vehicles can be positive only
#if we relocate from station i to j using a large vehicle
relocation_LCV_positive_if_large_is_1=m.addConstrs((relocation_LCV[i, j]<=1000*is_large_relocation[i, j] 
                                        for i  in stations for j in stations), name="relocation_LCV_positive_if_small_is_1")

#auxiliary variable satisfied_demand
satisfied_demand = m.addVars(stations,vtype=GRB.INTEGER, lb = 0, name="satisfied_demand")

#constraint that states that the maximum number of bikes relocated from station i is the starting inventory at station i
m.addConstrs((quicksum(relocation_SCV[i, j]+relocation_LCV[i, j] for j in stations)<=(starting_inventory.loc[i,"count"]) for i in stations), name="maximum_relocation")

#satisfied demand smaller than demand
m.addConstrs((satisfied_demand[i]<=demand.loc[i, "count"] for i in stations),
                 name="satisfied_demand_vs_demand")

#satisfied demand smaller than post-rebalancing inventory
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"]
              + quicksum(relocation_SCV[j,i]+relocation_LCV[j,i] for j in stations) 
              -quicksum(relocation_SCV[i,j]+relocation_LCV[i,j] for j in stations) 
              + replenishment.loc[i,"count"] for i in stations),
             name = "satisfied_demand_vs_inventory_after_rebalancing")

#objective
m.setObjective(4*satisfied_demand.sum()
               -1*quicksum(relocation_LCV[i,j]*distances.loc[i,j] for i in stations for j in stations)
               -0.5*quicksum(relocation_SCV[i,j]*distances.loc[i,j] for i in stations for j in stations)- 
               10*(number_small_vehicles+number_large_vehicles)-80*get_small_vehicles               
               ,GRB.MAXIMIZE)

In [19]:
m.optimize()
printSolution()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 951510 rows, 1899576 columns and 5220554 nonzeros
Model fingerprint: 0xf6654850
Model has 474722 quadratic constraints
Variable types: 0 continuous, 1899576 integer (949443 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 3e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [7e-05, 8e+01]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 2067 rows and 1378 columns (presolve time = 5s) ...
Presolve removed 47508 rows and 93605 columns (presolve time = 10s) ...
Presolve removed 47532 rows and 93605 columns (presolve time = 15s) ...
Presolve removed 76085 rows and 122158 columns (presolve time = 20s) ...
Presolve removed

     0     2 75571.1573    0  200 75285.3991 75571.1573  0.38%     - 2181s
     1     4 75569.2073    1  187 75285.3991 75571.1573  0.38%   147 2188s
     3     8 75565.1646    2  186 75285.3991 75571.1573  0.38%  95.3 2206s
     7    12 75564.4456    3  210 75285.3991 75571.1573  0.38%  84.6 2224s
    11    16 75564.7627    3  196 75285.3991 75571.1573  0.38%  71.9 2231s
    15    20 75564.1300    4  209 75285.3991 75571.1573  0.38%  66.1 2243s
    19    24 75483.7886    5  205 75285.3991 75571.1573  0.38%  62.3 2249s
    23    28 75561.9239    5  201 75285.3991 75571.1573  0.38%  55.3 2258s
    27    32 75560.8070    6  209 75285.3991 75571.1573  0.38%  56.4 2275s
    31    36 75560.0540    6  197 75285.3991 75571.0397  0.38%  55.0 2286s
    35    40 75560.5531    7  204 75285.3991 75571.0397  0.38%  51.9 2298s
    39    50 75479.1403    7  207 75285.3991 75571.0397  0.38%  47.9 2316s
    49    62 75559.5569   10  202 75285.3991 75571.0397  0.38%  46.2 2345s
    61    71 75557.9322  

GurobiError: Out of memory

**Is the rebalancing operation still marginally profitable?**

*Insert your answer: Even though the model I have designed was too complex to be solved by Gurobi within the local memory constraints, the best solution reached by Gurobi before the memory ran out had a net revenue of 75285.4, which is still much higher than net revenue without rebalancing, 72024. Thus, the rebalancing operation is still marginally profitable.* 


# Q6. Qualitative Discussion (15 pt)

**Please provide detailed answers to the following questions.**

### Q6.0 
**What are the main limitations of our current modeling approach? What other aspects of the rebalancing would you account for?**

*Insert your answer: One limitation of our current modelling approach is that we assume all rides are worth the same. However, each station most likely has its specific average revenue for customer rides, and to improve our net revenue we should account for this in our optimization model. Moreover, to make sure the rebalancing operation is finished in time, it is important to implement some constraints on the maximum duration allowed for bike relocation for each vehicle. We might also want to extend the time horizon we are considering and look into how the relocation of bikes between stations affects our revenues after the peak hour.* 


### Q6.1. 
**In reality, you don't know in advance the exact number of user request in the next hour. How could you remedy this issue?**

*Insert your answer: We can attempt to predict the number of user requests in the next hour by training predictive machine learning algorithms on historical data. Provided we have enough data, we should be able to accurately predict demand at peak hours. We should try to look at demand for all possible transportation options, so that we get a clearer picture of the overall market, and then try to predict what percentage of this demand bike sharing services such as ours could capture.* 

### Q6.2.
One of your colleagues has further concerns about how the demand is estimated: *Our historical data only records the customers who are satisfied and complete their trip. Many of our potential customers walk out without even taking a bike!* 

**Explain the above claim. Is this an important issue? Are you currently over-estimating or under-estimating the value of rebalancing? How would you deal with this approach?** 

*Insert your answer: The above claim suggests that estimating demand using historical data on bike rides isn't accurate because it doesn't take into account potential customers who might have wanted to rent a bike at a certain time but didn't, perhaps because there were no bikes available at their preferred docking station and therefore walked away. This is a very valid concern because we have no way of knowing exactly how many bikes would have been rented at an empty docking station at a certain time, had there been more bikes available. This migth then cause us to underestimate demand for some docking stations, which would in turn underestimate the value of rebalancing. We could to deal with this issue in multiple ways. We might want to use a censored regression model (tobit), where the dependent variable is censored above a certain threshold. We could also try to directly see what demand would be at docking stations which are often empty by simply running experiments where we reloate more bikes to these stations until we know for sure that demand is fully satisfied (when a surplus arises). We could also try to design an online app where customers can easily book rides before heading to the docking station, perhaps even at a discount. By seeing how many people attempt to book rides at certain times, we can get a clearer sense of the real demand for bikes.* 


### Q6.3. 
In practice, rebalancing is not an easy operation (see the picture below). It requires to hire specialized labor and rent trucks.

<img src="citi-bike-truck.jpg" width="500">

**What other aspects can be incorporated into the optimization model to make it more realistic?**

*Insert your answer: To make the optimization model more realistic, there should certainly be a constraint on the maximum number of bikes that can be relcated in one haul, even for large capacity vehicles. There should also be constraints on the maximum number of vehicles that we can hire on a given day, since providers probably only have a limited fleet capacity or available drivers. Then, rather than having a constraint on the maximum number of rebalancings which a vehicle can make, I think it would be more realistic to have a constraint on the maximum amount of time a vehicle can operate for before the peak hour, and time would be determined by the number of kilometers travelled and the number of stops.*

### Q6.4 
**For which other operational decisions could Citibike utilize optimization models to improve its efficiency?**

*Insert your answer: The most obvious operational decision for which Citibike could utilize optimization models to improve its efficieny is the number of docking stations, their locations, and also their docking capacity. Even though this analysis should have been made before starting the bike sharing business, using optimization models is still very important for deciding where to open new docking stations and what their capacities should be. Optimization models could also be used for pricing decisions.* 


# Executive Summary Report (15 pts)

Write a 1 pager report summarizing your findings (11 pt font, 1 inch margin, pdf/doc format). This report should take the form of an executive summary that combines elements from your analysis and business recommendations. You can also suggest additional analysis that should be conducted to augment the current rebalancing model. Supporting evidence can be provided in the appendix or by referencing the questions of the workshop. The report will be evaluated along 3 dimensions: clarity, scientific validity, and practical relevance.